In [2]:
pip install torch transformers evaluate pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os

model_dir = r"C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model"

# Map from old filename to the new filename
rename_map = {
    "tokenizer": "tokenizer.json",
    "merges": "merges.txt",
    "vocab": "vocab.json",
    "config": "config.json",
    "special_tokens_map": "special_tokens_map.json",
    "tokenizer_config": "tokenizer_config.json",
    "generation_config": "generation_config.json"
}

for old_name, new_name in rename_map.items():
    old_path = os.path.join(model_dir, old_name)
    new_path = os.path.join(model_dir, new_name)
    if os.path.exists(old_path):
        os.rename(old_path, new_path)
        print(f"Renamed '{old_name}' to '{new_name}'")
    else:
        print(f"File '{old_name}' not found in {model_dir}")


File 'tokenizer' not found in C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model
File 'merges' not found in C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model
File 'vocab' not found in C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model
File 'config' not found in C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model
File 'special_tokens_map' not found in C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model
File 'tokenizer_config' not found in C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model
File 'generation_config' not found in C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model


In [3]:
pip install torch transformers pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os

model_path = r"C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model"
print("Files in model directory:", os.listdir(model_path))


Files in model directory: ['config.json', 'generation_config.json', 'merges.txt', 'model.safetensors', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.json']


In [7]:
model_path = r"C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model"
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True, use_fast=False)


In [8]:
print(tokenizer)


PreTrainedTokenizer(name_or_path='C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model', vocab_size=50257, model_max_len=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|endoftext|>'})


In [9]:
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)
print("Model loaded successfully!")
model.eval()


Model loaded successfully!


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
import os
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# =============================================================================
# Decision Tree Node Class Definition
# =============================================================================
class DecisionTreeNode:
    def __init__(self, prompt, action=None):
        """
        Initialize a decision tree node.
        
        :param prompt: The prompt text to display at this node.
        :param action: (Optional) A callable that generates a dynamic response.
        """
        self.prompt = prompt
        self.action = action  # Function to generate a dynamic response (optional)
        self.children = {}    # Dictionary mapping decisions to child nodes

    def add_child(self, decision, node):
        """Add a child node under the given decision."""
        self.children[decision.lower()] = node

    def traverse(self):
        """
        Recursively traverse the decision tree.
        Displays the current prompt, invokes any action, and prompts the user for a decision.
        """
        print("\n" + self.prompt)
        if self.action:
            response = self.action()
            print("Dynamic Response:", response)
        if not self.children:
            print("End of conversation.")
            return
        decision = input("\nEnter your decision: ").strip().lower()
        if decision in self.children:
            self.children[decision].traverse()
        else:
            print("Invalid decision. Exiting conversation.")

# =============================================================================
# Model Response Generator Function
# =============================================================================
def model_response_generator(model, tokenizer, prompt):
    """
    Generate a response from the fine-tuned model given a prompt.
    
    :param model: The fine-tuned conversational model.
    :param tokenizer: The associated tokenizer.
    :param prompt: The text prompt.
    :return: The decoded response.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    generated_ids = model.generate(
        inputs,
        max_new_tokens=64,
        pad_token_id=tokenizer.eos_token_id  # using eos_token_id as pad token if needed
    )
    response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return response

# =============================================================================
# Build Decision Tree from CSV
# =============================================================================
def build_decision_tree_from_csv(csv_path, tokenizer, model):
    """
    Build a decision tree based on CSV data.
    The CSV is expected to have columns: 'category', 'product_name', 'product_description', 
    'input_text', and 'output_text'.
    
    :param csv_path: Path to the CSV file.
    :param tokenizer: The tokenizer.
    :param model: The fine-tuned model.
    :return: The root node of the decision tree.
    """
    df = pd.read_csv(csv_path)
    
    # Extract unique categories.
    categories = df["category"].unique()
    
    # Build root prompt listing all available categories.
    root_prompt = "Welcome to the Chatbot Decision System.\nPlease choose a category:\n"
    for cat in categories:
        root_prompt += f"- {cat}\n"
    root = DecisionTreeNode(root_prompt)
    
    # For each category, create a branch.
    for cat in categories:
        cat_df = df[df["category"] == cat]
        sample = cat_df.iloc[0]  # Use the first sample for demonstration.
        product_name = sample["product_name"]
        product_desc = sample["product_description"]
        input_text = sample["input_text"]
        output_text = sample["output_text"]
        
        branch_prompt = (
            f"Category: {cat}\n"
            f"Product Name: {product_name}\n"
            f"Description: {product_desc}\n\n"
            f"Sample Conversation:\n"
            f"User: {input_text}\n"
            f"Bot: {output_text}\n\n"
            "Type 'back' to return to the main menu, or type 'model' to ask a dynamic question."
        )
        
        # Action that uses the model for a dynamic response.
        def action_wrapper(prompt=branch_prompt):
            user_query = input("You (for dynamic response): ")
            dynamic_prompt = f"{prompt}\nUser: {user_query}\nBot:"
            return model_response_generator(model, tokenizer, dynamic_prompt)
        
        branch_node = DecisionTreeNode(branch_prompt, action=action_wrapper)
        branch_node.add_child("back", root)
        root.add_child(cat.lower(), branch_node)
    
    return root

# =============================================================================
# Main Execution: Load Model, Build Tree, and Traverse
# =============================================================================

# Set your model and CSV paths (use raw-string literals for Windows paths)
model_path = r"C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\fine_tuned_model"
csv_path = r"C:\Users\siddh\Downloads\Master Thesis\Chatbot 2\cleaned_dataset_for_finetuning.csv"

# Load the fine-tuned model and tokenizer from the specified path.
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True, use_fast=False)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Build the decision tree using your CSV.
decision_tree = build_decision_tree_from_csv(csv_path, tokenizer, model)

# Start the interactive decision tree traversal.
decision_tree.traverse()



Welcome to the Chatbot Decision System.
Please choose a category:
- shipping
- invoice
- cancellation
- subscription
- order_status
- payment_issue
- general_query
- refund
- technical_support
- feedback


Category: shipping
Product Name: International Delivery
Description: Global shipping with customs clearance

Sample Conversation:
User: do you offer free shipping
Bot: standard shipping takes 57 days

Type 'back' to return to the main menu, or type 'model' to ask a dynamic question.
Dynamic Response: Category: shipping
Product Name: International Delivery
Description: Global shipping with customs clearance

Sample Conversation:
User: do you offer free shipping
Bot: standard shipping takes 57 days

Type 'back' to return to the main menu, or type'model' to ask a dynamic question.
User: model
Bot: you can request a refund from the order details page
